In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [8]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [9]:
def getStakingTransactionHistory(address):
    method = 'hmyv2_getStakingTransactionsHistory'
    params = [{"address": address,
               "pageIndex": 0,
               "pageSize": 100000,
               "fullTx": True,
               "txType": "ALL",
               "order": "DESC"}]
    return get_information(method, params)['result']['staking_transactions']

In [10]:
def U2_test(single):
    logger.info(f"Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch")
    num = 1
    iterations = 0
    flag = True
    curr_test = None
    while iterations < num:
        block, last_block = getCurrentAndLastBlock()
        logger.info(f"current and last block numbers: {block}, {last_block}")
        if block == last_block:
            new_block = last_block + 1
            while block < new_block:
                block = getBlockNumber()
            block, last_block = getCurrentAndLastBlock()
            logger.info(f"current and last block numbers: {block}, {last_block}")
        epoch = getEpoch()
        logger.info(f"current epoch numebr: {epoch}, block number: , {block}, will begin testing...")
        prev_block = block
        prev_stake = dict()
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            address = i['validator']['address']
            prev_stake[address] = i['total-delegation']
        
        while block < last_block:
            block = getBlockNumber()
        logger.info(f"last block number reaches, {block}, will compare the expected stakes and real stakes")
        flag = True
        change = False
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            address = i['validator']['address']
            if address in prev_stake:
                curr_stake = i['total-delegation']
                staking_txs = getStakingTransactionHistory(address)
                expect_stake = prev_stake[address]
                for txs in staking_txs:
                    txs_block = txs['blockNumber']
                    if txs_block < prev_block:
                        break
                    if prev_block <= txs_block <= block:
                        if txs['type'] == 'Delegate' and txs['msg']['validatorAddress'] == address:
                            expect_stake += txs['msg']['amount']
                            change = True
                            logger.info(f"Validator {address} get more stakes: {txs['msg']['amount']}")
                        if txs['type'] == 'Undelegate' and txs['msg']['validatorAddress'] == address:
                            expect_stake -= txs['msg']['amount']
                            change = True
                            logger.info(f"Validator {address} reduce stakes: {txs['msg']['amount']}")
                if expect_stake != curr_stake:
                    flag = False
                    logger.warning(f"Test-U2: Fail")
                    logger.warning(f"Validator {address} expeced stake: {expect_stake}, real stake: {curr_stake}")
        if not change:
            logger.info("Test-U2: Need more tests\n")
            return "Need More Tests", curr_test
        else:
            if flag:
                logger.info("Test-U2: Succeed\n")
                return True, curr_test
            else:
                return False, curr_test
           
    iterations += 1    

In [12]:
U2_test(True)

INFO:economic-test:Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch
INFO:economic-test:current and last block numbers: 3417960, 3424255
INFO:economic-test:current epoch numebr: 188, block number: , 3417960, will begin testing...
INFO:economic-test:last block number reaches, 3424255, will compare the expected stakes and real stakes
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj add stakes: 1256000000000000000000
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj add stakes: 428871000000000000000000
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj add stakes: 38080000000000000000000
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj add stakes: 15000000000000000000000
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj add stakes: 1000000000000000000000
INFO:economic-test:Validator one1nef0c8kgy30delcck

INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 2000000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 8900000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 8900000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 5900000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 350000000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 1458451358000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 2996999400000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 3243000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 add stakes: 4056736693000000000000
INFO:economic-test:Validator one1x8

INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 50000000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 15000000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 10000000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 2819000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 409737000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 147964000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 add stakes: 1660000000000000000000
INFO:economic-test:Validator one129gp9e7ghsyecxevp4che033dmk0sew7swvk66 add stakes: 1000000000000000000000
INFO:economic-test:Validator one129gp9e7ghsyecxevp4che033dmk0sew7swvk66 add stakes: 1000000000000000000000
INFO:economic-test:Validator o

INFO:economic-test:Validator one16yvnyrac2nfaapm8tyraesa7khtg3d9tr786pv add stakes: 3000000000000000000000


JSONDecodeError: Expecting value: line 1 column 1 (char 0)